# Reading and writing files
In many use cases, you will want your python code to read/write from/to files stored on your local hard drive.  
Here are a few important points to consider when working with files:
* do I need to read the entire dataset/file into memory?
    * remember the table about the time a computer takes to perform various action. Accessing the hard drive is among the slower operations. 
      Reading an entire file when you only need the first few lines will be costly
    * if you are reading a very large file, then having the entire file in memory at once may overburden your computer
* are there concurrency issues?
    * if another software (or even your code if you have messed up) writes to a file you are currently reading, you could run into trouble.

Whether it is for reading or for writing, operations with files occur using **file objects** (sometimes also refferred-to as file **handles**).
File objects are created using the `open()` function, and closed using the object's `.close()` method.  
Here is a basic example:

```python
# Open a file in a given "mode" (e.g. read, writte or append).
file = open(filename, mode)

# Do something with the file...

# When you are done, don't forget to close the file.
file.close()
```

> <span style="color:red">Remember to close the files you have opened, or you will encounter errors if you want to access this file again further down in your code.</span>

<br>

### File opening modes
When using the `open()` function, a **mode** can be passed as argument to the function. This specifies the type of access you will have on the file. For instance, the `'r'` mode will only allow to read the content of a file, and will not allow writing to it (this is useful to avoir accidental writing to the file).

There are several more modes of opening files.
* `'r'`: open file in read-only mode.
* `'w'`: open file in write-only mode, **overwriting** existing file with the same name.
* `'a'`: open file in write-only mode, **appending** to existing file with the same name.
* `'rb'`, `'wb'`, `'ab'`: same as `'r'`, `'w'` and `'a'`, but reading/writing to/from binary files (such as `.zip` or `.bmp` image files). 
  The content is read/written as bytes objects without any decoding.

See `help(open)` for a full list of modes and details about them.

<br>

## Reading from files
To start reading a file, one creates a **file object** using the `'r'` mode of the `open` function.  
Then, reading lines can be done simply by iterating over the file object in a `for` loop or using the `.readline()` method of file objects.

In [1]:
reading_handle = open('fresh_fruits.txt' , 'r')
i = 0
for line in reading_handle:
    print('line', i, ':', line)
    i += 1
    
reading_handle.close()


line 0 : passionfruit

line 1 : oranges

line 2 : apples

line 3 : grapefruit (whole and segments)

line 4 : pointed sticks


<br>

### End-of-line characters
As you can see in the example above, there are additionnal empty lines in between our prints. This is because the lines are read from the file with their **end-of-line** characters, which generally is `\n` .  
To avoid this kind of issue, one typically uses the `.strip()` method of strings, which removes any whitespace or *end-of-line* character at the start or end of the string.

Here is an illustratin of using `.strip()` when reading content from a file, this time using a `while` loop:

In [2]:
reading_handle = open('fresh_fruits.txt', 'r')
i = 0
line = reading_handle.readline()

# When the file has been entirely read, readline() returns an empty string and the while loop will end.
# In python a non empty string evalutes to "True", and therefore we can use "while line" as a shortcut for "while line != '' ".
while line:
    print('line', i, ':', line.strip())    # Note: here we use the "strip()" method of "str" to remove the trailing "\n" (carriage return) of each line.
    line = reading_handle.readline()       # Don't forget this or you will have an infinite loop.
    i += 1
    
reading_handle.close()

line 0 : passionfruit
line 1 : oranges
line 2 : apples
line 3 : grapefruit (whole and segments)
line 4 : pointed sticks


<br>

And here is yet another way to read the fruity content of our file, this time using the `readlines()` function instead of `readline()`.  
As its name suggests, `readlines()` reads more than one line at a time (by default, all lines in the file).


<br>

And here is yet another way to read the fruity content of our file, this time using the `readlines()` function instead of `readline()`.
As its name suggests, `readlines()` reads more than one line at a time (by default, all lines in the file).

**Question:** while our examples using `readline()` or `readlines()` work equally well, there can be important implications in using one or the other of these functions, especially when dealing with large files.  
can you think of a drawback of using `readlines()` ?

In [3]:
reading_handle = open('fresh_fruits.txt' , 'r')
entire_file = reading_handle.readlines()
for i, line in enumerate(entire_file):         # Reminder: the enumerate() function allows to generate (index, value) tuples for any iterable object.
    print('line', i, ':', line.strip())

reading_handle.close()

line 0 : passionfruit
line 1 : oranges
line 2 : apples
line 3 : grapefruit (whole and segments)
line 4 : pointed sticks


<br>

**Answer:** using `readlines()` will (by default) load the entire file in memory, and this can be problematic when working with large files as is often the case in bioinformatics.  
Always consider the file sizes you are dealign with when using `readlines()`.

<br>

### Opening files with context managers
Now that you understand the basics of opening and closing a file, we can show you the actual "pythonic", recommended, way to deal with files:

In [4]:
with open('fresh_fruits.txt', 'r') as file:
    for i, line in enumerate(file):
        print('line', i, ':', line.strip())
        
# Wait, did we just forget to close our file ?

line 0 : passionfruit
line 1 : oranges
line 2 : apples
line 3 : grapefruit (whole and segments)
line 4 : pointed sticks


<br>

As you might have noticed in the example above, there is no explicit call to `close()`. So how does our file get closed then?

The magic happens in the `with ... as ...` construct, which starts a **context manager** code block in python.  
**Context managers** are code blocks that have special functions associated with them that execute automatically at the 
start and end of the context manager block.  
In the case of `with open() as ...:`, the special function executed at the start of the block is opening the file, and 
the special function executed at the end of the code block is closing the file handle.

Using the `with open() as ...:` context manager has several advantages:
* shorter, abstracted code: you don't have to worry about the implementation details of opening and closing a file.
* guarantee that a file will always be closed, no matter what happens (e.g. if an error occurs).

Context managers are not covered in this course, but you can look them up if you are interested.

<br>

### Mini Exercise:
Read the content of `fresh_fruits.txt` using a context manager and a while loop. Make sure to remove empty lines between print outputs.

<br>

## Writing to files
Writing to a file is achieved in pretty much the same way as reading from it, but the opening mode is now `'w'`.  
And instead of reading lines, we now `print()` them to the file.

In [5]:
with open("shopping_list.txt", 'w') as writing_handle:
    print("onion", file=writing_handle)
    print( 34 , "potato" , file=writing_handle)
    print("shrubbery", file=writing_handle)
    print("tomato sauce", file=writing_handle)


By passing the file object (or file handle) to the `file` argument of the `print()` function, we now print to the file rather than to our terminal.
> **Reminder**: the `'w'` mode overwrites the opened file - *i.e.*, if you use it on an existing file, its original content is lost.  
> **Pro tip:** you can open more than one file using a single `with` statement:
```python
with open('input.txt', 'r') as in_file, open('output.txt', 'w') as out_file:
    do_something()
```

<br>

### Mini Exercise:
Write some code to read the content of the `shopping_list.txt` file we just created, in order to check that the writing did work properly. Make sure that no white space is printed between lines.

<br>

## Exercises: 3.1, 3.2 and 3.3